In [ ]:
!pip install opencv-contrib-python

In [ ]:
!pip uninstall opencv-python opencv-python-headless -y


In [ ]:
!pip install opencv-contrib-python


In [ ]:
!pip install opencv-contrib-python==4.5.5.64


In [ ]:
#!/usr/bin/env python

import sys
import os
import numpy as np
import cv2
from face_recognition_system.videocamera import VideoCamera
from face_recognition_system.detectors import FaceDetector
import face_recognition_system.operations as op


def get_images(frame, faces_coord, shape):
    """ Perform transformation on original and face images. """
    if shape == "rectangle":
        faces_img = op.cut_face_rectangle(frame, faces_coord)
        frame = op.draw_face_rectangle(frame, faces_coord)
    faces_img = op.normalize_intensity(faces_img)
    faces_img = op.resize(faces_img)
    return frame, faces_img


def add_person(people_folder, shape):
    """ Function to add pictures of a person """
    person_name = input('What is the name of the new person: ').lower()
    folder = people_folder + person_name
    if not os.path.exists(folder):
        input("I will now take 20 pictures. Press ENTER when ready.")
        os.mkdir(folder)
        video = VideoCamera()
        detector = FaceDetector('face_recognition_system/frontal_face.xml')
        counter = 1
        timer = 0
        cv2.namedWindow('Video Feed', cv2.WINDOW_AUTOSIZE)
        cv2.namedWindow('Saved Face', cv2.WINDOW_NORMAL)
        while counter < 21:
            frame = video.get_frame()
            if frame is None or frame.size == 0:
                print("Error: Empty frame received. Please check the camera.")
                break
            face_coord = detector.detect(frame)
            if len(face_coord):
                frame, face_img = get_images(frame, face_coord, shape)
                if timer % 100 == 5:
                    cv2.imwrite(folder + '/' + str(counter) + '.jpg', face_img[0])
                    print('Images Saved:' + str(counter))
                    counter += 1
                    cv2.imshow('Saved Face', face_img[0])

            cv2.imshow('Video Feed', frame)
            cv2.waitKey(1)
            timer += 5
        video.release()
        cv2.destroyAllWindows()
    else:
        print("This name already exists.")
        sys.exit()


def recognize_people(people_folder, shape):
    """ Start recognizing people in a live stream with your webcam """
    try:
        people = [person for person in os.listdir(people_folder)]
    except:
        print("Have you added at least one person to the system?")
        sys.exit()

    print("These are the people in the Recognition System:")
    for person in people:
        print("- " + person)

    print(30 * '-')
    print("   POSSIBLE RECOGNIZERS TO USE")
    print(30 * '-')
    print("1. EigenFaces")
    print("2. FisherFaces")
    print("3. LBPHFaces")
    print(30 * '-')

    choice = check_choice()

    detector = FaceDetector('face_recognition_system/frontal_face.xml')
    recognizer = None
    threshold = None

    # Creating face recognizer based on user choice
    if choice == 1:
        if hasattr(cv2.face, 'EigenFaceRecognizer_create'):
            recognizer = cv2.face.EigenFaceRecognizer_create()
            threshold = 4000
        else:
            print("EigenFaceRecognizer is not available.")
            sys.exit()
    elif choice == 2:
        if hasattr(cv2.face, 'FisherFaceRecognizer_create'):
            recognizer = cv2.face.FisherFaceRecognizer_create()
            threshold = 300
        else:
            print("FisherFaceRecognizer is not available.")
            sys.exit()
    elif choice == 3:
        if hasattr(cv2.face, 'LBPHFaceRecognizer_create'):
            recognizer = cv2.face.LBPHFaceRecognizer_create()
            threshold = 80
        else:
            print("LBPHFaceRecognizer is not available.")
            sys.exit()

    images = []
    labels = []
    labels_people = {}
    for i, person in enumerate(people):
        labels_people[i] = person
        for image in os.listdir(people_folder + person):
            images.append(cv2.imread(people_folder + person + '/' + image, 0))
            labels.append(i)

    try:
        recognizer.train(images, np.array(labels))
    except Exception as e:
        print(f"\nOpenCV Error: {e}\n")
        sys.exit()

    video = VideoCamera()
    while True:
        frame = video.get_frame()

        # Check if the frame is valid
        if frame is None or frame.size == 0:
            print("Error: Empty frame received. Please check the camera.")
            break

        faces_coord = detector.detect(frame)
        if len(faces_coord):
            frame, faces_img = get_images(frame, faces_coord, shape)
            for i, face_img in enumerate(faces_img):
                if cv2.__version__ == "3.1.0":
                    collector = cv2.face.MinDistancePredictCollector()
                    recognizer.predict(face_img, collector)
                    conf = collector.getDist()
                    pred = collector.getLabel()
                else:
                    pred, conf = recognizer.predict(face_img)

                print("Prediction:", pred)
                print('Confidence:', round(conf))
                print('Threshold:', threshold)
                if conf < threshold:
                    cv2.putText(frame, labels_people[pred].capitalize(),
                                (faces_coord[i][0], faces_coord[i][1] - 2),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1,
                                cv2.LINE_AA)
                else:
                    cv2.putText(frame, "Unknown",
                                (faces_coord[i][0], faces_coord[i][1]),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1,
                                cv2.LINE_AA)

        cv2.putText(frame, "Press 'q' to exit", (5, frame.shape[0] - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Check for 'q' key
            break

    video.release()
    cv2.destroyAllWindows()


def check_choice():
    """ Check if the choice is valid """
    is_valid = 0
    while not is_valid:
        try:
            choice = int(input('Enter your choice [1-3]: '))
            if choice in [1, 2, 3]:
                is_valid = 1
            else:
                print(f"'{choice}' is not an option.\n")
        except ValueError as error:
            print(f"{error} is not an option.")
    return choice


if __name__ == '__main__':
    print(30 * '-')
    print("   POSSIBLE ACTIONS")
    print(30 * '-')
    print("1. Add person to the recognizer system")
    print("2. Start recognizer")
    print("3. Exit")
    print(30 * '-')

    CHOICE = check_choice()

    PEOPLE_FOLDER = "face_recognition_system/people/"
    SHAPE = "rectangle"

    if CHOICE == 1:
        if not os.path.exists(PEOPLE_FOLDER):
            os.makedirs(PEOPLE_FOLDER)
        add_person(PEOPLE_FOLDER, SHAPE)
    elif CHOICE == 2:
        os.system("sudo modprobe bcm2835-v4l2")  # Required for Raspi-Camera Module
        recognize_people(PEOPLE_FOLDER, SHAPE)
    elif CHOICE == 3:
        sys.exit()
